In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys, os
path_current = '/home/huzuntao/PycharmProjects/MPP_Powersystem/'
path_ = os.getcwd()
if path_current not in sys.path:
    sys.path.insert(1, '/home/huzuntao/PycharmProjects/MPP_Powersystem/')
elif path_ not in sys.path:
    sys.path.insert(1, path_)

import pandapower as pp
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)
bus5 = pp.create_bus(net, vn_kv=110.)
bus6 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus4, bus5, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus2, bus3, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus6, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
p_load_1 = 10
p_load_2 = 30
pp.create_load(net, bus2, p_mw=p_load_1, controllable=False)
pp.create_load(net, bus4, p_mw=p_load_2/2, controllable=False)
pp.create_load(net, bus5, p_mw=p_load_2/2, controllable=False)
pp.create_load(net, bus6, p_mw=p_load_2/2, controllable=False)
# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)

costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.bus["min_vm_pu"] = 0.96
net.bus["max_vm_pu"] = 1.04
net.line["max_loading_percent"] = 100
om, ppopt, raw = pp.runopp(net, delta=1e-16, RETURN_RAW_DER=1)

## pipscopf.py

In [2]:
import numpy, torch
from torch import ones, zeros, pi, exp, conj, cat, tensor
Inf = tensor(float('inf'))
from pypower_pych_gpu.torch_utils import find
from pypower_pych_gpu.csr_real_imag import csr_imag
from pypower_pych_gpu.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_pych_gpu.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
from pypower_pych_gpu.idx_cost import MODEL, PW_LINEAR, NCOST
from pypower_pych_gpu.idx_gen import GEN_BUS, PG, QG, VG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN
from pypower_pych_gpu.makeYbus import makeYbus
from pypower_pych_gpu.opf_consfcn import opf_consfcn
from pypower_pych_gpu.opf_costfcn import opf_costfcn

# from pandapower.pypower.util import sub2ind

from pypower_pych_gpu.opf_hessfcn import opf_hessfcn
from pypower_pych_gpu.pips import pips

import warnings
warnings.filterwarnings("ignore")

# pip.py
import torch
# from numpy import array, Inf, any, isnan, ones, r_, finfo, \
#     zeros, dot, absolute, log, flatnonzero as find
# from numpy.linalg import norm

# from scipy.sparse import vstack, hstack, eye, csr_matrix as sparse
# from scipy.sparse.linalg import spsolve
from torch import tensor, any, isnan, ones, cat, finfo, \
    zeros, dot, absolute, log, arange
from pypower_pych_gpu.torch_utils import find
from torch import asarray
from torch.linalg import norm
Inf = tensor(float('inf'))

from pypower_pych_gpu.torch_utils import vstack, hstack
from torch import eye
from pypower_pych_gpu.torch_utils import sparse, issparse
# from cupyx.scipy.sparse.linalg import spsolve
from torch.linalg import solve

from pandapower.pypower.pipsver import pipsver

EPS = finfo(float).eps
    # Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


Using cuda device


In [3]:
x0_init=None
out_opt=None

##----- initialization -----
## optional output
if out_opt is None:
    out_opt = {}

## options
verbose = ppopt['VERBOSE']
feastol = ppopt['PDIPM_FEASTOL']
gradtol = ppopt['PDIPM_GRADTOL']
comptol = ppopt['PDIPM_COMPTOL']
costtol = ppopt['PDIPM_COSTTOL']
max_it  = ppopt['PDIPM_MAX_IT']
max_red = ppopt['SCPDIPM_RED_IT']
init = ppopt['INIT']
step_control = (ppopt['OPF_ALG'] == 565)  ## OPF_ALG == 565, PIPS-sc
if feastol == 0:
    feastol = ppopt['OPF_VIOLATION']
opt = {  'feastol': feastol,
         'gradtol': gradtol,
         'comptol': comptol,
         'costtol': costtol,
         'max_it': max_it,
         'max_red': max_red,
         'step_control': step_control,
         'cost_mult': 1,
         'verbose': verbose  }

## unpack data TODO: move the data to GPU
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = tensor(ppc["baseMVA"], device=device), tensor(ppc["bus"], device=device), tensor(ppc["gen"], device=device), \
 tensor(ppc["branch"], device=device), tensor(ppc["gencost"], device=device)
vv, _, nn, _ = om.get_idx()
# vv, nn = tensor(vv), tensor(nn)
## problem dimensions
nb = bus.shape[0]          ## int: number of buses
nl = branch.shape[0]       ## int: number of branches
ny = om.getN('var', 'y')   ## int: number of piece-wise linear costs

## linear constraints
A, l, u = om.linear_constraints()
if isinstance(A, numpy.ndarray):
    A = tensor(A, device=device)
if isinstance(l, numpy.ndarray):
    l = tensor(l, device=device)
if isinstance(u, numpy.ndarray):
    u = tensor(u, device=device)


## bounds on optimization vars
x0, xmin, xmax = om.getv()
x0, xmin, xmax = tensor(x0, device=device), tensor(xmin, device=device), tensor(xmax, device=device)

## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)
if ('torch' not in str(type(Ybus))) or ('torch' not in str(type(Yf))) or ('torch' not in str(type(Yt))):
    raise ValueError('Ybus/Yf/Yt must be a torch tensor')
else:
    Ybus = Ybus.to(device)
    Yf = Yf.to(device)
    Yt = Yt.to(device)

## try to select an interior initial point if init is not available from a previous powerflow
if init != "pf":
    ll, uu = xmin.clone(), xmax.clone()
    ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
    uu[xmax ==  Inf] =  1e10
    x0 = (ll + uu) / 2
    Varefs = bus[bus[:, BUS_TYPE] == REF, VA] * (pi / 180)
    ## angles set to first reference angle
    x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = Varefs[0]
    if ny > 0:
        pass

if x0_init is not None:
    x0 = x0_init

## find branches with flow limits
il = find((branch[:, RATE_A] != 0) & (branch[:, RATE_A].real < 1e10))
Yf_on, Yt_on = (Yf.to_dense()[il, :]).to_sparse(layout=torch.sparse_csr), (Yt.to_dense()[il, :]).to_sparse(layout=torch.sparse_csr)

nl2 = len(il)           ## number of constrained lines


cp = om.get_cost_params()
N, Cw, H, dd, rh, kk, mm = \
    cp["N"], cp["Cw"], cp["H"], cp["dd"], cp["rh"], cp["kk"], cp["mm"]
print(f"N: {N}, Cw: {Cw}, H: {H}, dd: {dd}, rh: {rh}, kk: {kk}, mm: {mm}")
print(f"cp: {cp}")

N: [], Cw: [], H: [], dd: [], rh: [], kk: [], mm: []
cp: {'N': array([], shape=(0, 16), dtype=float64), 'Cw': array([], dtype=float64), 'H': array([], shape=(0, 0), dtype=float64), 'dd': array([], dtype=float64), 'rh': array([], dtype=float64), 'kk': array([], dtype=float64), 'mm': array([], dtype=float64)}


In [4]:
cp = [tensor(item).to(device) for item in (N, Cw, H, dd, rh, kk, mm)]
pack_para = (0, baseMVA, bus, gen, branch, gencost, il, vv, nn, ny, cp)
if issparse(N) and N.nnz > 0:
    print('N is sparse')
##-----  run opf  -----
f_fcn = lambda x, return_hessian=False: opf_costfcn(x, pack_para, return_hessian)
gh_fcn = lambda x: opf_consfcn(x, pack_para, Ybus, Yf_on, Yt_on, ppopt, il)
hess_fcn = lambda x, lmbda, cost_mult: opf_hessfcn(x, lmbda, pack_para, Ybus, Yf_on, Yt_on, ppopt, il, cost_mult)


## pip.py

In [5]:
if isinstance(f_fcn, dict):  ## problem dict
    p = f_fcn
    f_fcn = p['f_fcn']
    x0 = p['x0']
    if 'opt' in p: opt = p['opt']
    if 'hess_fcn' in p: hess_fcn = p['hess_fcn']
    if 'gh_fcn' in p: gh_fcn = p['gh_fcn']
    if 'xmax' in p: xmax = p['xmax']
    if 'xmin' in p: xmin = p['xmin']
    if 'u' in p: u = p['u']
    if 'l' in p: l = p['l']
    if 'A' in p: A = p['A']
# print(f"isinstance(f_fcn, dict): {isinstance(f_fcn, dict)}; x0 shape: {x0.shape}")

nx = x0.shape[0]                       # int: number of variables
nA = A.shape[0] if A is not None else 0 # int: number of original linear constr

# default argument values
if l is None or len(l) == 0: l = -Inf * ones(nA, device=device)
if u is None or len(u) == 0: u =  Inf * ones(nA, device=device)
if xmin is None or len(xmin) == 0: xmin = -Inf * ones(x0.shape[0], dtype=x0.dtype).to(device)
if xmax is None or len(xmax) == 0: xmax =  Inf * ones(x0.shape[0], dtype=x0.dtype).to(device)
if gh_fcn is None:
    nonlinear = False
    gn = tensor([], device=device)
    hn = tensor([], device=device)
else:
    nonlinear = True

if opt is None: opt = {}
# options
if "feastol" not in opt:
    opt["feastol"] = 1e-06
if "gradtol" not in opt:
    opt["gradtol"] = 1e-06
if "comptol" not in opt:
    opt["comptol"] = 1e-06
if "costtol" not in opt:
    opt["costtol"] = 1e-06
if "max_it" not in opt:
    opt["max_it"] = 150
if "max_red" not in opt:
    opt["max_red"] = 20
if "step_control" not in opt:
    opt["step_control"] = False
if "cost_mult" not in opt:
    opt["cost_mult"] = 1
if "verbose" not in opt:
    opt["verbose"] = 0
opt["verbose"] = 1
# initialize history
hist = []

# constants
xi = 0.99995
sigma = 0.1
z0 = 1
alpha_min = 1e-8
rho_min = 0.95
rho_max = 1.05
mu_threshold = 1e-5

# initialize
i = 0                       # iteration counter
converged = False           # flag
eflag = False               # exit flag

# add var limits to linear constraints
eyex = eye(nx, nx, dtype=torch.float64, device=device)
print(f"A: {A}; eyex shape: {eyex.shape}; nx : {nx}")
AA = eyex if A is None else vstack([eyex, A], "csr")
ll = cat([xmin, l])
uu = cat([xmax, u])

# split up linear constraints
ieq = find( absolute(uu - ll) <= EPS )
igt = find( (uu >=  1e10) & (ll > -1e10) )
ilt = find( (ll <= -1e10) & (uu <  1e10) )
ibx = find( (absolute(uu - ll) > EPS) & (uu < 1e10) & (ll > -1e10) )
# zero-sized sparse matrices unsupported
# print(f'ieq_: {ieq};AA shape: {AA.shape}')
Ae = AA[ieq, :] if len(ieq) else None
if len(ilt) or len(igt) or len(ibx):
    idxs = [(1, ilt), (-1, igt), (1, ibx), (-1, ibx)]
    Ai = vstack([sig * AA[idx, :] for sig, idx in idxs if len(idx)], 'csr')
else:
    Ai = None
be = uu[ieq]
bi = cat([uu[ilt], -ll[igt], uu[ibx], -ll[ibx]])
x = x0
f, df = f_fcn(x)                 # cost
f = f * opt["cost_mult"]
df = df * opt["cost_mult"]
if nonlinear:
    hn, gn, dhn, dgn = gh_fcn(x)        # nonlinear constraints
    h = hn if Ai is None else cat([hn, Ai.to(torch.float64).matmul(x) - bi])  # inequality constraints
    g = gn if Ae is None else cat([gn, Ae.to(torch.float64).matmul(x) - be])
    if (dhn is None) and (Ai is None):
        dh = None
    elif dhn is None:
        dh = Ai.T
    elif Ai is None:
        dh = dhn
    else:
        dh = hstack([dhn, Ai.T])

    if (dgn is None) and (Ae is None):
        dg = None
    elif dgn is None:
        dg = Ae.T
    elif Ae is None:
        dg = dgn
    else:
        dg = hstack([dgn, Ae.T])
else:
    h = -bi if Ai is None else Ai * x - bi        # inequality constraints
    g = -be if Ae is None else Ae * x - be        # equality constraints
    dh = None if Ai is None else Ai.T     # 1st derivative of inequalities
    dg = None if Ae is None else Ae.T     # 1st derivative of equalities

# some dimensions
neq = g.shape[0]           # int: number of equality constraints
niq = h.shape[0]           # int: number of inequality constraints
neqnln = gn.shape[0]       # int: number of nonlinear equality constraints
niqnln = hn.shape[0]       # int: number of nonlinear inequality constraints
nlt = len(ilt)             # int: number of upper bounded linear inequalities
ngt = len(igt)             # int: number of lower bounded linear inequalities
nbx = len(ibx)             # int: number of doubly bounded linear inequalities

# initialize gamma, lam, mu, z, e
gamma = 1                  # barrier coefficient
lam = zeros(neq, dtype=torch.float64, device=device)
z = z0 * ones(niq, dtype=torch.float64, device=device)
mu = z0 * ones(niq, dtype=torch.float64, device=device)
k = find(h < -z0)
z[k] = -h[k].double()
k = find((gamma / z) > z0)
mu[k] = gamma / z[k]
e = ones(niq, dtype=torch.float64, device=device)

# check tolerance
f0 = f
if opt["step_control"]:
    L = f + dot(lam, g) + dot(mu, h + z) - gamma * sum(log(z))

Lx = df.clone()
Lx = Lx + dg.matmul(lam) if dg is not None else Lx
Lx = Lx + dh.matmul(mu)  if dh is not None else Lx

maxh = zeros(1, device=device) if len(h) == 0 else max(h)

gnorm = norm(g, Inf) if len(g) else 0.0
lam_norm = norm(lam, Inf) if len(lam) else 0.0
mu_norm = norm(mu, Inf) if len(mu) else 0.0
znorm = norm(z, Inf) if len(z) else 0.0
feascond = \
    max([gnorm, maxh]) / (1 + max([norm(x, Inf), znorm]))
gradcond = \
    norm(Lx, Inf) / (1 + max([lam_norm, mu_norm]))
compcond = dot(z, mu) / (1 + norm(x, Inf))
costcond = absolute(f - f0) / (1 + absolute(f0))

# save history
hist.append({'feascond': feascond, 'gradcond': gradcond,
    'compcond': compcond, 'costcond': costcond, 'gamma': gamma,
    'stepsize': 0, 'obj': f / opt["cost_mult"], 'alphap': 0, 'alphad': 0})

if opt["verbose"]: # pragma: no cover
    s = '-sc' if opt["step_control"] else ''
    v = pipsver('all')
    print('Python Interior Point Solver - PIPS%s, Version %s, %s' %
                (s, v['Version'], v['Date']))
    if opt['verbose'] > 1:
        print(" it    objective   step size   feascond     gradcond     "
              "compcond     costcond  ")
        print("----  ------------ --------- ------------ ------------ "
              "------------ ------------")
        print("%3d  %12.8g %10s %12g %12g %12g %12g" %
            (i, (f / opt["cost_mult"]), "",
             feascond, gradcond, compcond, costcond))

if feascond < opt["feastol"] and gradcond < opt["gradtol"] and \
    compcond < opt["comptol"] and costcond < opt["costtol"]:
    converged = True
    if opt["verbose"]:
        print("Converged!")

A: None; eyex shape: torch.Size([16, 16]); nx : 16
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


In [10]:
dh.to_sparse_coo()

In [83]:
# Convert the sparse matrix to COO format
coo = dh.to_sparse_coo().coalesce()

# Print the sparse matrix in the desired format
for i, (r, c) in enumerate(coo.indices().T):
    print(f"({r.item()}, {c.item()})\t{coo.values()[i].item()}")

(0, 3)	398.4826354980469
(0, 8)	-398.4826354980469
(1, 2)	379.50726318359375
(1, 3)	-398.4826354980469
(1, 7)	-379.50726318359375
(1, 8)	398.4826354980469
(2, 1)	379.50726318359375
(2, 2)	-379.50726318359375
(2, 4)	-379.50726318359375
(2, 6)	-379.50726318359375
(2, 7)	379.50726318359375
(2, 9)	379.50726318359375
(3, 0)	379.50726318359375
(3, 1)	-379.50726318359375
(3, 5)	-379.50726318359375
(3, 6)	379.50726318359375
(4, 0)	-379.50726318359375
(4, 5)	379.50726318359375
(5, 4)	379.50726318359375
(5, 9)	-379.50726318359375
(6, 3)	13643.8583984375
(6, 8)	15325.3203125
(7, 2)	-797.57080078125
(7, 3)	-13681.1142578125
(7, 7)	802.0514526367188
(7, 8)	-15282.5068359375
(7, 10)	1.0
(7, 19)	-1.0
(8, 1)	-797.57080078125
(8, 2)	802.0514526367188
(8, 4)	802.0514526367188
(8, 6)	802.0514526367188
(8, 7)	-797.57080078125
(8, 9)	-799.3409423828125
(8, 11)	1.0
(8, 20)	-1.0
(9, 0)	-797.57080078125
(9, 1)	802.0514526367188
(9, 5)	802.0514526367188
(9, 6)	-797.57080078125
(9, 12)	1.0
(9, 21)	-1.0
(10, 0)	

In [84]:
Ai

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  1.],
        [-0., -0., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,
         -0., -0.],
        [-

### Iteration loop

In [6]:
# do Newton iterations
while (not converged) and (i < opt["max_it"]):
    # update iteration counter
    i += 1
    print(f"max it: {opt['max_it']}; i: {i}")

    # compute update step
    lmbda = {"eqnonlin": lam[range(neqnln)],
             "ineqnonlin": mu[range(niqnln)]}
    if nonlinear:
        if hess_fcn is None:
            print("pips: Hessian evaluation via finite differences "
                  "not yet implemented.\nPlease provide "
                  "your own hessian evaluation function.")
        Lxx = hess_fcn(x, lmbda, opt["cost_mult"])
    else:
        _, _, d2f = f_fcn(x, True)      # cost
        Lxx = d2f * opt["cost_mult"]
    rz = arange(len(z))
    zinvdiag = sparse((1.0 / z, (rz, rz))) if len(z) else None
    rmu = arange(len(mu))
    mudiag = sparse((mu, (rmu, rmu))) if len(mu) else None
    dh_zinv = None if dh is None else dh.matmul(zinvdiag)
    M = Lxx if dh is None else Lxx + dh_zinv.matmul(mudiag).matmul(dh.T)
    N = Lx if dh is None else Lx + dh_zinv.matmul((mudiag.matmul(h.to(torch.float64)) + gamma*e))

    Ab = M.to_sparse_csr if dg is None else vstack([
        hstack([M, dg]),
        hstack([dg.T, zeros((neq, neq), dtype=torch.float64)])
    ])
    bb = cat([-N, -g])

    dxdlam = solve(Ab.to_dense(), bb).double()

    if any(isnan(dxdlam)):
        if opt["verbose"]:
            print('\nNumerically Failed\n')
        eflag = -1
        break

    dx = dxdlam[:nx]
    dlam = dxdlam[nx:nx + neq]
    dz = -h - z if dh is None else -h.to(torch.float64) - z - dh.T.matmul(dx)
    dmu = -mu if dh is None else -mu + zinvdiag.matmul( (gamma * e - mudiag.matmul(dz)))

    # do the update
    k = find(dz < 0.0)
    alphap = min([xi * min(z[k] / -dz[k]), 1]) if len(k) else 1.0
    k = find(dmu < 0.0)
    alphad = min([xi * min(mu[k] / -dmu[k]), 1]) if len(k) else 1.0
    x = x + alphap * dx
    z = z + alphap * dz
    lam = lam + alphad * dlam
    mu = mu + alphad * dmu
    if niq > 0:
        gamma = sigma * dot(z, mu) / niq

    # evaluate cost, constraints, derivatives
    f, df = f_fcn(x)             # cost
    f = f * opt["cost_mult"]
    df = df * opt["cost_mult"]
    if nonlinear:
        hn, gn, dhn, dgn = gh_fcn(x)                   # nln constraints
#            g = gn if Ai is None else r_[gn, Ai * x - bi] # ieq constraints
#            h = hn if Ae is None else r_[hn, Ae * x - be] # eq constraints
        h = hn if Ai is None else cat([hn, Ai.to(torch.float64).matmul(x) - bi])  # inequality constraints
        g = gn if Ae is None else cat([gn, Ae.to(torch.float64).matmul(x) - be])
        # h = hn if Ai is None else cat([hn.reshape(len(hn),), Ai * x - bi]) # ieq constr
        # g = gn if Ae is None else cat([gn, Ae * x - be])  # eq constr

        if (dhn is None) and (Ai is None):
            dh = None
        elif dhn is None:
            dh = Ai.T
        elif Ai is None:
            dh = dhn
        else:
            dh = hstack([dhn, Ai.T])

        if (dgn is None) and (Ae is None):
            dg = None
        elif dgn is None:
            dg = Ae.T
        elif Ae is None:
            dg = dgn
        else:
            dg = hstack([dgn, Ae.T])
    else:
        h = -bi if Ai is None else Ai.matmul(x) - bi    # inequality constraints
        g = -be if Ae is None else Ae.matmul(x) - be    # equality constraints
        # 1st derivatives are constant, still dh = Ai.T, dg = Ae.T

    Lx = df
    Lx = Lx + dg.matmul(lam) if dg is not None else Lx
    Lx = Lx + dh.matmul(mu)  if dh is not None else Lx

    if len(h) == 0:
        maxh = zeros(1)
    else:
        maxh = max(h)

    gnorm = norm(g, Inf) if len(g) else 0.0
    lam_norm = norm(lam, Inf) if len(lam) else 0.0
    mu_norm = norm(mu, Inf) if len(mu) else 0.0
    znorm = norm(z, Inf) if len(z) else 0.0
    feascond = \
        max([gnorm, maxh]) / (1 + max([norm(x, Inf), znorm]))
    gradcond = \
        norm(Lx, Inf) / (1 + max([lam_norm, mu_norm]))
    compcond = dot(z, mu) / (1 + norm(x, Inf))
    costcond = float(absolute(f - f0) / (1 + absolute(f0)))

    hist.append({'feascond': feascond, 'gradcond': gradcond,
        'compcond': compcond, 'costcond': costcond, 'gamma': gamma,
        'stepsize': norm(dx), 'obj': f / opt["cost_mult"],
        'alphap': alphap, 'alphad': alphad})

    if opt["verbose"] > 1:
        print("%3d  %12.8g %10.5g %12g %12g %12g %12g" %
            (i, (f / opt["cost_mult"]), norm(dx), feascond, gradcond,
             compcond, costcond))

    if feascond < opt["feastol"] and gradcond < opt["gradtol"] and \
        compcond < opt["comptol"] and costcond < opt["costtol"]:
        converged = True
        if opt["verbose"]:
            print("Converged!")
    else: #TODO alpha and alphad are too small
        if any(isnan(x)) or (alphap < alpha_min) or \
            (alphad < alpha_min) or (gamma < EPS) or (gamma > 1.0 / EPS):
            if opt["verbose"]:
                print("Numerically failed.")
            eflag = -1
            break
        f0 = f

        if opt["step_control"]:
            L = f + dot(lam, g) + dot(mu, (h + z)) - gamma * sum(log(z))

max it: 150; i: 1


RuntimeError: expected scalar type ComplexDouble but found Double

In [36]:
print(f"Lxx: {Lxx}")

Lxx: tensor([[ 6.0829e+05, -6.0829e+05,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -5.1728e-12,  1.1710e-11,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-6.0829e+05,  9.5812e+05, -3.4982e+05,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  6.1959e-12, -3.5811e-12, -9.0949e-12,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00, -3.4982e+05,  1.2790e+06, -3.4982e+05,  0.0000e+00,
         -5.7933e+05,  0.0000e+00, -9.0949e-12,  1.2108e-11, -9.0949e-12,
          0.0000e+00,  6.1959e-12,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -3.4982e+05,  6.9965e+05, -3.4982e+05,
          0.0000e+00,  0.0000e+00,  0.0000e+00, -9.0949e-12,  1.8253e-11,
         -9.0949e-12,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.000

In [37]:
print(f"dxdlam:{dxdlam}")

dxdlam:tensor([ 1.0625e-17, -1.0842e-02,  9.0286e-03, -8.8349e-02, -1.3601e-01,
        -2.7796e-02,  4.0629e-15,  2.2318e-02, -8.7540e-03, -3.9595e-02,
        -5.7197e-02, -2.2954e-02, -4.9134e+02,  6.3846e+00,  6.8734e+00,
        -2.0122e+01,  1.8035e+01,  1.6087e+01,  1.0319e+01,  1.2953e+02,
         1.8916e+02,  7.0142e+01,  1.3747e-08, -4.2665e+01, -4.0244e-08,
        -1.6770e+01, -2.2253e+01, -4.4564e+00,  1.6526e-04, -1.1539e+03])


In [44]:
# print(f"Ab:{Ab.to_sparse_csr()}\nbb:{bb}")
# Convert the sparse matrix to COO format
coo = Ab.T.to_sparse_coo().coalesce()

# Print the sparse matrix in the desired format
for i, (r, c) in enumerate(coo.indices().T):
    print(f"({r.item()}, {c.item()})\t{coo.values()[i].item()}")

(0, 0)	608334.4586080383
(0, 1)	-608334.4586080384
(0, 6)	-95.61320495605986
(0, 7)	91.44357299805858
(0, 16)	361.701416015625
(0, 17)	-361.701416015625
(0, 22)	-171.14651489257812
(0, 23)	171.14651489257812
(0, 28)	1.0
(1, 0)	-608334.4586080384
(1, 1)	958193.5279525565
(1, 2)	-349859.0693445183
(1, 6)	95.61320495606088
(1, 7)	-167.17160034180046
(1, 8)	75.72801208495184
(1, 16)	-361.701416015625
(1, 17)	629.6283569335938
(1, 18)	-267.9269714355469
(1, 22)	171.14651489257812
(1, 23)	-297.9217224121094
(1, 24)	126.77519989013672
(2, 1)	-349859.0693445183
(2, 2)	1279080.7074834527
(2, 3)	-349859.0693445183
(2, 5)	-579362.5687906015
(2, 7)	75.72801208495184
(2, 8)	-227.2594757079957
(2, 9)	75.72801208495184
(2, 11)	75.8034439086976
(2, 17)	-267.9269714355469
(2, 18)	880.3314819335938
(2, 19)	-267.9269714355469
(2, 21)	-344.4775390625
(2, 23)	126.77519989013672
(2, 24)	-416.5470886230469
(2, 25)	126.77519989013672
(2, 27)	162.99668884277344
(3, 2)	-349859.0693445183
(3, 3)	699718.138689036

In [43]:
Ab.shape

torch.Size([30, 30])

In [45]:
bb

tensor([ 5.4932e-04, -5.4932e-04, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -3.7871e+00,  3.7852e+00, -1.4257e-03, -1.1206e-03,
        -5.7125e-04, -3.8815e-04, -2.0000e+01, -1.0000e+01, -0.0000e+00,
        -0.0000e+00,  4.9144e+02, -1.8502e+00,  4.0000e+01, -1.5000e+01,
        -1.5000e+01, -1.5000e+01, -1.6802e+01,  1.9885e+01,  4.1577e+00,
         2.9935e+00,  1.4968e+00,  1.1642e+00, -0.0000e+00, -0.0000e+00],
       dtype=torch.float64)

In [46]:
Lx + dh_zinv.matmul((mudiag.matmul(h.to(torch.float64)) + gamma*e))

tensor([-4.2607e+03,  4.4650e+03, -2.9238e+02,  3.0037e+02, -3.7525e+02,
         1.6303e+02,  2.6763e+07, -2.6754e+07,  9.5632e+03,  6.9665e+03,
         3.4467e+03,  2.1676e+03,  7.9713e+00,  5.2640e+00, -6.9401e-09,
         2.0317e-08])

In [49]:
Lx

tensor([-3.1543e+02,  5.2274e+02, -2.9670e+02,  3.0112e+02, -3.7497e+02,
         1.6325e+02,  1.4931e+04, -1.5603e+04, -7.1576e+02, -9.3400e+02,
        -5.0326e+02, -2.2190e+02,  9.9030e+00,  4.9515e+00, -6.9401e-09,
         2.0317e-08])

In [50]:
dh_zinv.matmul((mudiag.matmul(h.to(torch.float64)) + gamma*e))

tensor([-3.9453e+03,  3.9423e+03,  4.3125e+00, -7.5000e-01, -2.8125e-01,
        -2.1875e-01,  2.6748e+07, -2.6738e+07,  1.0279e+04,  7.9005e+03,
         3.9500e+03,  2.3895e+03, -1.9316e+00,  3.1250e-01,  0.0000e+00,
         0.0000e+00])

In [52]:
dg

tensor([[ 371.4802, -367.7521,    0.0000,    0.0000,    0.0000,    0.0000,
         -170.9520,  178.8311,    0.0000,    0.0000,    0.0000,    0.0000,
            1.0000,    0.0000],
        [-371.4802,  636.6406, -273.9060,    0.0000,    0.0000,    0.0000,
          170.9520, -312.5503,  123.1152,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000],
        [   0.0000, -268.8885,  878.8698, -242.5683,    0.0000, -327.8716,
            0.0000,  133.7193, -364.7171,  144.6392,    0.0000,  169.9256,
            0.0000,    0.0000],
        [   0.0000,    0.0000, -265.6591,  490.7067, -237.3682,    0.0000,
            0.0000,    0.0000,   95.8391, -248.1220,  126.2447,    0.0000,
            0.0000,    0.0000],
        [   0.0000,    0.0000,    0.0000, -248.1384,  237.3682,    0.0000,
            0.0000,    0.0000,    0.0000,  103.4828, -126.2447,    0.0000,
            0.0000,    0.0000],
        [   0.0000,    0.0000, -339.3047,    0.0000,    0.0000,  327.8716,
            0.0

In [71]:
coo = dh.to_sparse_coo().coalesce()

# Print the sparse matrix in the desired format
for i, (r, c) in enumerate(coo.indices().T):
    print(f"({r.item()}, {c.item()})\t{coo.values()[i].item()}")


(0, 3)	3719.8564453125
(0, 8)	2905.451904296875
(1, 2)	-3076.91064453125
(1, 3)	-3719.8564453125
(1, 7)	-3845.75634765625
(1, 8)	-2905.451904296875
(2, 1)	16289.1103515625
(2, 2)	3076.91064453125
(2, 4)	9791.4658203125
(2, 6)	15569.2275390625
(2, 7)	3845.75634765625
(2, 9)	10526.4873046875
(3, 0)	7773.71533203125
(3, 1)	-16289.1103515625
(3, 5)	7086.015625
(3, 6)	-15569.2275390625
(4, 0)	-7773.71533203125
(4, 5)	-7086.015625
(5, 4)	-9791.4658203125
(5, 9)	-10526.4873046875
(6, 3)	7309.56982421875
(6, 8)	8982.7646484375
(7, 2)	4553.4462890625
(7, 3)	-7286.76171875
(7, 7)	6202.8623046875
(7, 8)	-8931.783203125
(7, 10)	1.0
(7, 19)	-1.0
(8, 1)	5316.720703125
(8, 2)	-4521.10888671875
(8, 4)	5010.76025390625
(8, 6)	6832.85400390625
(8, 7)	-6091.81494140625
(8, 9)	3449.999755859375
(8, 11)	1.0
(8, 20)	-1.0
(9, 0)	2458.002197265625
(9, 1)	-3702.2333984375
(9, 5)	3961.814697265625
(9, 6)	-5311.51611328125
(9, 12)	1.0
(9, 21)	-1.0
(10, 0)	-2068.157470703125
(10, 5)	-3611.999755859375
(10, 13)	1.